In [ ]:
import copy
import numpy as np
import scipy
import scipy.interpolate
import scipy.optimize
from tqdm import tqdm

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import palettable
import descartes

In [ ]:
import verdict

In [ ]:
import stained_glass.generate as generate
import stained_glass.idealized as idealized
import stained_glass.stats as stats
import stained_glass.sample as sample

# Parameters

In [ ]:
# Number of sightlines
n = 10000
sidelength = 600.
annuli = np.array([ 20., 100., 200., 300. ])
edges_log = np.logspace( -1., np.log10( sidelength), 64 )
edges = np.linspace( 0., sidelength, 64 )

In [ ]:
xs = edges[:-1] + 0.5 * ( edges[1] - edges[0] )

In [ ]:
xs_log = 10.**( np.log10( edges_log[:-1] ) + 0.5 * ( np.log10( edges_log[1] ) - np.log10( edges_log[0] ) ) )

In [ ]:
n_per_bin = round( n / (edges_log.size - 1 ) / 2 )

# Setup idealized projections

In [ ]:
ips = {}
all_length_scales = {}

### Parameters

In [ ]:
# Main halo
r_vir = 250.
m_vir = 1e12

# Filament
filament_val = 3e7
dx = -400.
theta_b = 100.

# Clumps
dr = 0.1
r_clumps = 10.**np.arange( -1, 2 + dr, dr )
f_covs = np.arange( 0.1, 1.0, 0.1 )
clump_val = 5e7

### Control w no clumps

In [ ]:
ip = idealized.IdealizedProjection( sidelength )
ip.add_nfw_nopatch(
    center = (0., 0.),
    r_vir = r_vir,
    m_vir = m_vir,
    n_annuli = 64,
)
edge_val = min( ip.struct_values )

# # Satellite
# # ip.add_sphere(
# #     c = (-250., 50.),
# #     r = r_sat,
# #     value = value_sat,
# #     n_annuli = 64,
# # )

# # Filament
# ip.add_curve(
#     v1 = (0., 0.),
#     v2 = (dx, 60.),
#     theta_a = 20.,
#     theta_b = theta_b,
#     value = filament_val,
# )
# length_scales = {}
# width = 40.
# n_concentric = 40
# ip.add_concentric_structures(
#     ip.structs[-1],
#     value = filament_val,
#     n_concentric = n_concentric,
#     dr = width / n_concentric,
#     dv = - ( filament_val - edge_val ) / n_concentric
# )

length_scales = {}
# length_scales['long'] = np.sqrt( dx**2. + 60.**2. )
length_scales['halo'] = r_vir
length_scales['annuli'] = 100.
# length_scales['satellite'] = r_sat

ips['fcov0.0'] = { 'rclump0.0': ip }
all_length_scales['fcov0.0'] = { 'rclump0.0': length_scales }

### Varying clump radii

In [ ]:
pbar = tqdm( total=f_covs.size*r_clumps.size, position=0, leave=True)

for f_cov in f_covs:
    fcov_key = 'fcov{:.3g}'.format( f_cov )
    fcov_ips = {}
    fcov_ls = {}
    
    for r_clump in r_clumps:
        rclump_key = 'rclump{:.3g}'.format( r_clump )
        
        # Main halo
        ip = idealized.IdealizedProjection(sidelength)
        ip.add_nfw_nopatch(
            center = (0., 0.),
            r_vir = r_vir,
            m_vir = m_vir,
            n_annuli = 64,
        )
        edge_val = min( ip.struct_values )

#         # Filament
#         ip.add_curve(
#             v1 = (0., 0.),
#             v2 = (dx, 60.),
#             theta_a = 20.,
#             theta_b = theta_b,
#             value = filament_val,
#         )
#         width = 40.
#         n_concentric = 40
#         ip.add_concentric_structures(
#             ip.structs[-1],
#             value = filament_val,
#             n_concentric = n_concentric,
#             dr = width / n_concentric,
#             dv = - ( filament_val - edge_val ) / n_concentric
#         )

        # Clumps
        ip.add_clumps_nopatch(
            r_clump = r_clump,
            c = (0., 0.),
            r_area = r_vir,
            fcov = f_cov,
            value = clump_val,
        )
        length_scales = {}
        # length_scales['long'] = np.sqrt( 300.**2. + 60.**2. )
        length_scales['halo'] = r_vir
        length_scales['clump'] = r_clump
        length_scales['annuli'] = 100.
        
        # Generate projection
        ip.generate_idealized_projection()
        
        pbar.update( 1 )

        # structs, values = copy.copy( ip.structs ), copy.copy( ip.struct_values )
        # for i, (struct, value) in enumerate( zip( *[ structs, values ] ) ):
        #     width = 10.
        #     n_concentric = 10
        #     ip.add_concentric_structures(
        #         ip.structs[-1],
        #         value = filament_val,
        #         n_concentric = n_concentric,
        #         dr = width / n_concentric,
        #         dv = - ( filament_val - 1. ) / n_concentric
        #     )
        
        fcov_ips[rclump_key] = ip
        fcov_ls[rclump_key] = length_scales
    ips[fcov_key] = fcov_ips
    all_length_scales[fcov_key] = fcov_ls
    
pbar.close()

# Calculate Metrics

### Generate Paired Coordinates

In [ ]:
v_edges = np.array([ ip.ip_values.min(), ip.ip_values.max() ])

In [ ]:
pair_sampler = sample.PairSampler( ip.sidelength, edges_log, v_edges )
dr_coords1, dr_coords2 = pair_sampler.generate_pair_sampling_coords(
    n_per_bin = n_per_bin,
)

In [ ]:
pair_coords = np.concatenate([ np.concatenate( dr_coords1 ), np.concatenate( dr_coords2 ) ])

### Calculate pair-sampled, weighted TPCF

In [ ]:
pbar = tqdm( total=f_covs.size*r_clumps.size + 1, position=0, leave=True)

fcovs = {}
tpcfs = {}
for fcov_key, f_cov_ips in ips.items():
    tpcfs_fcov = {}
    fcovs_fcov = {}
    
    for rclump_key, ip in f_cov_ips.items():
        
        # Get data
        ip.set_sightlines( pair_coords )
        ws = ip.evaluate_sightlines()

        tpcf, edges = stats.weighted_tpcf(
            pair_coords,
            ws,
            edges_log,
        )
        
        pbar.update( 1 )
    
        tpcfs_fcov[rclump_key] = tpcf
        fcovs_fcov[rclump_key] = ( ws >= 0.999*clump_val ).sum() / float( ws.size )
        
    tpcfs[fcov_key] = tpcfs_fcov
    fcovs[fcov_key] = fcovs_fcov
pbar.close()

### Calculate delta(TPCF=0.25,0.5,0.75)

In [ ]:
intercepts = {}
for intercept in [ 0.25, 0.5, 0.75 ]:
    intercepts_part = {}
    for i, (fcov_key, f_cov_tpcfs) in enumerate( tpcfs.items() ):

        half_intercepts_fcov = {}

        for j, (rclump_key, tpcf) in enumerate( f_cov_tpcfs.items() ):  

            interp_fn = scipy.interpolate.interp1d( np.log10( xs_log ), tpcf )

            def root_fn( delta ):

                return interp_fn( delta ) - intercept

            try:
                sol = scipy.optimize.root_scalar( root_fn, method='brentq', bracket=[ np.log10( xs_log[0] ), np.log10( 5e2 ) ], )
            except:
                continue

            half_intercepts_fcov[rclump_key] = sol.root

        intercepts_part[fcov_key] = half_intercepts_fcov   
    intercepts[intercept] = intercepts_part
intercepts = verdict.Dict( intercepts )

# Plot

### Setup

In [ ]:
ncols = len( r_clumps )
nrows = len( f_covs ) + 1

In [ ]:
def label_or_not( label, do_label ):
    
    if do_label:
        return label
    
    return None

In [ ]:
colors = palettable.matplotlib.Viridis_4.mpl_colors[1:][::-1]
arc_color = palettable.cartocolors.qualitative.Pastel_10.mpl_colors[2]

In [ ]:
matplotlib.rcParams['xtick.major.size'] = 10
matplotlib.rcParams['xtick.major.width'] = 2
matplotlib.rcParams['xtick.minor.size'] = 5
matplotlib.rcParams['xtick.minor.width'] = 1.4
matplotlib.rcParams['ytick.major.size'] = 10
matplotlib.rcParams['ytick.major.width'] = 2
matplotlib.rcParams['ytick.minor.size'] = 5
matplotlib.rcParams['ytick.minor.width'] = 1.4

matplotlib.rc('xtick', labelsize=16) 
matplotlib.rc('ytick', labelsize=16) 

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

### Same Axis TPCF

In [ ]:
fig = plt.figure( figsize=(12,8), facecolor='w' )
ax = plt.gca()

for i, (fcov_key, f_cov_tpcfs) in enumerate( tpcfs.items() ):
    
    for j, (rclump_key, tpcf) in enumerate( f_cov_tpcfs.items() ):  
    
        # TPCFs
        ax.plot(
            xs_log,
            tpcf,
            linewidth = 6,
            color = 'k',
            label = i,
            zorder = -1,
        )
        
        ax.axvline(
            10.**half_intercepts[fcov_key][rclump_key]
        )
        
ax.axhline(
    0.5,
)
    
# Axis tweaks
ax.set_xlim( 0.9, xs_log[-1] )
ax.set_xscale( 'log' )
ax.set_ylim( -0.5, 1. )

### Plot delta(TPCF=0.5) vs Rclump

In [ ]:
fig = plt.figure( figsize=(12,12), facecolor='w' )
ax = plt.gca()

for i, (fcov_key, f_cov_his) in enumerate( half_intercepts.items() ):
    
    # Reformat
    r_clumps_plot = []
    his_plot = []
    for j, (rclump_key, f_cov_hi) in enumerate( f_cov_his.items() ):  
        r_clumps_plot.append( float( rclump_key[6:] ) )
        his_plot.append( 10.**f_cov_hi )
        
    ax.scatter(
        r_clumps_plot,
        his_plot,
        s = 70,
        color = palettable.matplotlib.Viridis_10.mpl_colormap( float( fcov_key[4:] ) ),
    )
    
ax.plot(
    [ 0.1, 1e2 ],
    [ 0.1, 1e2 ],
    color = 'k',
    linestyle = '--',
    linewidth = 3,
)
    
ax.set_aspect( 'equal' )
    
ax.set_xscale( 'log' )
ax.set_yscale( 'log' )

ax.set_xlabel( r'$R({\rm clump})$ (kpc)', fontsize=22 )
ax.set_ylabel( r'$\delta(\Xi=0.5)$ (kpc)', fontsize=22 )

fig.savefig('./characteristic_r.pdf', bbox_inches='tight')